In [1]:
#https://www.machinelearningplus.com/nlp/topic-modeling-gensim-python/
import re
import numpy as np
import pandas as pd
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
from sklearn.datasets import fetch_20newsgroups
cats = ['rec.autos', 'rec.motorcycles',
        'sci.crypt', 'sci.electronics', 
        'sci.med', 'sci.space',
        'talk.politics.guns', 'talk.religion.misc',
        'rec.sport.baseball', 'rec.sport.hockey']
cats = None
df = fetch_20newsgroups(subset = 'all', categories=cats)

from pprint import pprint
pprint(list(df.target_names))

['alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'comp.windows.x',
 'misc.forsale',
 'rec.autos',
 'rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.crypt',
 'sci.electronics',
 'sci.med',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast',
 'talk.politics.misc',
 'talk.religion.misc']


In [3]:
df.filenames.shape

(18846,)

In [4]:
df.target.shape

(18846,)

In [5]:
# Convert to list
data = df.data

# Remove Emails
data = [re.sub('\S*@\S*\s?', '', sent) for sent in data]

# Remove new line characters
data = [re.sub('\s+', ' ', sent) for sent in data]

# Remove distracting single quotes
data = [re.sub("\'", "", sent) for sent in data]

pprint(data[:1])

<input>:5: DeprecationWarning: invalid escape sequence \S
<input>:8: DeprecationWarning: invalid escape sequence \s
<input>:5: DeprecationWarning: invalid escape sequence \S
<input>:8: DeprecationWarning: invalid escape sequence \s
<input>:5: DeprecationWarning: invalid escape sequence \S
<input>:8: DeprecationWarning: invalid escape sequence \s
<ipython-input-5-1eed9f39b362>:5: DeprecationWarning: invalid escape sequence \S
  data = [re.sub('\S*@\S*\s?', '', sent) for sent in data]
<ipython-input-5-1eed9f39b362>:8: DeprecationWarning: invalid escape sequence \s
  data = [re.sub('\s+', ' ', sent) for sent in data]


['From: Mamatha Devineni Ratnam Subject: Pens fans reactions Organization: '
 'Post Office, Carnegie Mellon, Pittsburgh, PA Lines: 12 NNTP-Posting-Host: '
 'po4.andrew.cmu.edu I am sure some bashers of Pens fans are pretty confused '
 'about the lack of any kind of posts about the recent Pens massacre of the '
 'Devils. Actually, I am bit puzzled too and a bit relieved. However, I am '
 'going to put an end to non-PIttsburghers relief with a bit of praise for the '
 'Pens. Man, they are killing those Devils worse than I thought. Jagr just '
 'showed you why he is much better than his regular season stats. He is also a '
 'lot fo fun to watch in the playoffs. Bowman should let JAgr have a lot of '
 'fun in the next couple of games since the Pens are going to beat the pulp '
 'out of Jersey anyway. I was very disappointed not to see the Islanders lose '
 'the final regular season game. PENS RULE!!! ']


In [6]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(data))

print(data_words[:1])

[['from', 'mamatha', 'devineni', 'ratnam', 'subject', 'pens', 'fans', 'reactions', 'organization', 'post', 'office', 'carnegie', 'mellon', 'pittsburgh', 'pa', 'lines', 'nntp', 'posting', 'host', 'po', 'andrew', 'cmu', 'edu', 'am', 'sure', 'some', 'bashers', 'of', 'pens', 'fans', 'are', 'pretty', 'confused', 'about', 'the', 'lack', 'of', 'any', 'kind', 'of', 'posts', 'about', 'the', 'recent', 'pens', 'massacre', 'of', 'the', 'devils', 'actually', 'am', 'bit', 'puzzled', 'too', 'and', 'bit', 'relieved', 'however', 'am', 'going', 'to', 'put', 'an', 'end', 'to', 'non', 'pittsburghers', 'relief', 'with', 'bit', 'of', 'praise', 'for', 'the', 'pens', 'man', 'they', 'are', 'killing', 'those', 'devils', 'worse', 'than', 'thought', 'jagr', 'just', 'showed', 'you', 'why', 'he', 'is', 'much', 'better', 'than', 'his', 'regular', 'season', 'stats', 'he', 'is', 'also', 'lot', 'fo', 'fun', 'to', 'watch', 'in', 'the', 'playoffs', 'bowman', 'should', 'let', 'jagr', 'have', 'lot', 'of', 'fun', 'in', 'the

In [7]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
print(trigram_mod[bigram_mod[data_words[0]]])

['from', 'mamatha_devineni_ratnam', 'subject', 'pens', 'fans', 'reactions', 'organization', 'post', 'office', 'carnegie_mellon_pittsburgh_pa', 'lines', 'nntp_posting_host', 'po_andrew_cmu_edu', 'am', 'sure', 'some', 'bashers', 'of', 'pens', 'fans', 'are', 'pretty', 'confused', 'about', 'the', 'lack', 'of', 'any', 'kind', 'of', 'posts', 'about', 'the', 'recent', 'pens', 'massacre', 'of', 'the', 'devils', 'actually', 'am', 'bit', 'puzzled', 'too', 'and', 'bit', 'relieved', 'however', 'am', 'going', 'to', 'put', 'an', 'end', 'to', 'non', 'pittsburghers', 'relief', 'with', 'bit', 'of', 'praise', 'for', 'the', 'pens', 'man', 'they', 'are', 'killing', 'those', 'devils', 'worse', 'than', 'thought', 'jagr', 'just', 'showed', 'you', 'why', 'he', 'is', 'much', 'better', 'than', 'his', 'regular_season', 'stats', 'he', 'is', 'also', 'lot', 'fo', 'fun', 'to', 'watch', 'in', 'the', 'playoffs', 'bowman', 'should', 'let', 'jagr', 'have', 'lot', 'of', 'fun', 'in', 'the', 'next', 'couple', 'of', 'games'

In [8]:
# NLTK Stop words
import nltk; nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\aserg\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [9]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# !python -m spacy download en
nlp = spacy.load('en', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:1])

[['mamatha_devineni', 'ratnam', 'pen', 'fan', 'reaction', 'organization', 'post', 'office', 'carnegie_mellon', 'pittsburgh_pa', 'line', 'nntp_post', 'host', 'po_andrew', 'sure', 'basher', 'pen', 'fan', 'pretty', 'confused', 'lack', 'kind', 'post', 'recent', 'pen', 'massacre', 'devil', 'actually', 'bite', 'puzzled', 'bit', 'relieved', 'however', 'go', 'put', 'end', 'non', 'pittsburgher', 'relief', 'bit', 'praise', 'pen', 'man', 'kill', 'devil', 'worse', 'think', 'jagr', 'show', 'much', 'good', 'regular_season', 'stat', 'also', 'lot', 'fun', 'watch', 'playoff', 'bowman', 'let', 'jagr', 'lot', 'fun', 'next', 'couple', 'game', 'pen', 'go', 'beat', 'pulp', 'jersey', 'anyway', 'disappointed', 'see', 'islander', 'lose', 'final', 'regular_season', 'game', 'pen', 'rule']]


In [10]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1])

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 2), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 2), (12, 1), (13, 1), (14, 2), (15, 1), (16, 2), (17, 2), (18, 2), (19, 1), (20, 1), (21, 1), (22, 1), (23, 2), (24, 1), (25, 1), (26, 1), (27, 1), (28, 1), (29, 1), (30, 1), (31, 2), (32, 1), (33, 1), (34, 1), (35, 1), (36, 1), (37, 1), (38, 1), (39, 1), (40, 1), (41, 6), (42, 1), (43, 1), (44, 1), (45, 1), (46, 2), (47, 1), (48, 1), (49, 1), (50, 1), (51, 1), (52, 1), (53, 1), (54, 1), (55, 2), (56, 1), (57, 1), (58, 1), (59, 1), (60, 1), (61, 1), (62, 1), (63, 1), (64, 1), (65, 1)]]


In [11]:
# Human readable format of corpus (term-frequency)
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]

[[('actually', 1),
  ('also', 1),
  ('anyway', 1),
  ('basher', 1),
  ('beat', 1),
  ('bit', 2),
  ('bite', 1),
  ('bowman', 1),
  ('carnegie_mellon', 1),
  ('confused', 1),
  ('couple', 1),
  ('devil', 2),
  ('disappointed', 1),
  ('end', 1),
  ('fan', 2),
  ('final', 1),
  ('fun', 2),
  ('game', 2),
  ('go', 2),
  ('good', 1),
  ('host', 1),
  ('however', 1),
  ('islander', 1),
  ('jagr', 2),
  ('jersey', 1),
  ('kill', 1),
  ('kind', 1),
  ('lack', 1),
  ('let', 1),
  ('line', 1),
  ('lose', 1),
  ('lot', 2),
  ('mamatha_devineni', 1),
  ('man', 1),
  ('massacre', 1),
  ('much', 1),
  ('next', 1),
  ('nntp_post', 1),
  ('non', 1),
  ('office', 1),
  ('organization', 1),
  ('pen', 6),
  ('pittsburgh_pa', 1),
  ('pittsburgher', 1),
  ('playoff', 1),
  ('po_andrew', 1),
  ('post', 2),
  ('praise', 1),
  ('pretty', 1),
  ('pulp', 1),
  ('put', 1),
  ('puzzled', 1),
  ('ratnam', 1),
  ('reaction', 1),
  ('recent', 1),
  ('regular_season', 2),
  ('relief', 1),
  ('relieved', 1),
  ('rule'

In [12]:
def average_precision_at_k(theta, labels, k, dist_fn):
    dist = dist_fn(theta)
    np.fill_diagonal(dist, np.inf)
    idx = np.argpartition(dist, k-1)[:k,:]
    return np.mean(labels[idx] == labels[None,:])

In [13]:
from sklearn.metrics.pairwise import cosine_distances, euclidean_distances

def hellinger(theta):
    return euclidean_distances(np.sqrt(theta))

def dot(theta):
    return 1 - np.dot(theta, theta.transpose())

In [14]:
from numba import jit, prange, autojit

In [15]:
@jit(nopython = True, parallel = True)
def outer(X, fn):
    dist = np.empty((X.shape[0], X.shape[0]), dtype = np.float32)
    for i in prange(X.shape[0]):
        for j in prange(i):
            dist[j,i] = dist[i,j] = fn(X[i,:], X[j,:])           
    return dist

In [16]:
@jit(nopython = True)
def kl(p, q):
    return np.dot(p, np.log(q / p))

In [17]:
@jit(nopython = True)
def sym_kl(p, q):
    return kl(p, q) + kl(q, p)

In [18]:
@jit(nopython = True)
def js(p, q):
    m = (p + q) / 2
    return kl(p, m) / 2 + kl(q, m) / 2

In [19]:
@jit(nopython = True)
def bc(p, q):
    return -np.log(np.sum(np.sqrt(p * q)))

In [20]:
dist_fns = {'cosine' : cosine_distances,
            'eucl' : euclidean_distances,
            'hellinger' : hellinger,
            'dot' : dot,
            'sym_kl' : lambda x: outer(x, sym_kl),
            'js' : lambda x: outer(x, js),
            'bc' : lambda x: outer(x, bc)}

In [21]:
from gensim import matutils

def eval_topic_model(model, labels, k = 100, dist_fns = dist_fns):
    matrix = model.get_document_topics(corpus, minimum_probability=0)
    theta = matutils.corpus2dense(matrix, num_docs=len(corpus), num_terms=lda_model.num_topics, dtype = np.float32)
    theta = np.asarray(theta, dtype = np.float32)
    theta = theta.transpose()
    best_score = 0
    best_dist = None
    for (dist, f) in dist_fns.items():
        score = average_precision_at_k(theta, labels, 100, f)
        if score > best_score:
            best_score, best_dist = score, dist
    return best_score, best_dist

In [26]:
for num_topics in range(5, 60 + 1,5):
    lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=num_topics, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)
    score, dist = eval_topic_model(lda_model, df.target)
    print("%2i topics AP@100 = %.3f (%s distance)" % (num_topics, score, dist))

D:\Anaconda\lib\site-packages\sklearn\metrics\pairwise.py:258: RuntimeWarning: invalid value encountered in sqrt
  return distances if squared else np.sqrt(distances, out=distances)
D:\Anaconda\lib\site-packages\sklearn\metrics\pairwise.py:258: RuntimeWarning: invalid value encountered in sqrt
  return distances if squared else np.sqrt(distances, out=distances)


 5 topics AP@100 = 0.454 (bc distance)


D:\Anaconda\lib\site-packages\sklearn\metrics\pairwise.py:258: RuntimeWarning: invalid value encountered in sqrt
  return distances if squared else np.sqrt(distances, out=distances)
D:\Anaconda\lib\site-packages\sklearn\metrics\pairwise.py:258: RuntimeWarning: invalid value encountered in sqrt
  return distances if squared else np.sqrt(distances, out=distances)


10 topics AP@100 = 0.630 (js distance)


D:\Anaconda\lib\site-packages\sklearn\metrics\pairwise.py:258: RuntimeWarning: invalid value encountered in sqrt
  return distances if squared else np.sqrt(distances, out=distances)
D:\Anaconda\lib\site-packages\sklearn\metrics\pairwise.py:258: RuntimeWarning: invalid value encountered in sqrt
  return distances if squared else np.sqrt(distances, out=distances)


...

50 topics AP@100 = 0.577 (sym_kl distance)


D:\Anaconda\lib\site-packages\sklearn\metrics\pairwise.py:258: RuntimeWarning: invalid value encountered in sqrt
  return distances if squared else np.sqrt(distances, out=distances)
D:\Anaconda\lib\site-packages\sklearn\metrics\pairwise.py:258: RuntimeWarning: invalid value encountered in sqrt
  return distances if squared else np.sqrt(distances, out=distances)


55 topics AP@100 = 0.612 (js distance)


D:\Anaconda\lib\site-packages\sklearn\metrics\pairwise.py:258: RuntimeWarning: invalid value encountered in sqrt
  return distances if squared else np.sqrt(distances, out=distances)
D:\Anaconda\lib\site-packages\sklearn\metrics\pairwise.py:258: RuntimeWarning: invalid value encountered in sqrt
  return distances if squared else np.sqrt(distances, out=distances)


60 topics AP@100 = 0.690 (sym_kl distance)


In [22]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=20, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [23]:
pprint(lda_model.print_topics())

[(0,
  '0.033*"not" + 0.025*"would" + 0.021*"do" + 0.017*"be" + 0.015*"get" + '
  '0.014*"write" + 0.012*"go" + 0.012*"good" + 0.011*"think" + 0.011*"know"'),
 (1,
  '0.129*"drive" + 0.094*"card" + 0.072*"driver" + 0.036*"device" + '
  '0.029*"cpu" + 0.027*"scsi" + 0.027*"controller" + 0.026*"disk" + '
  '0.025*"board" + 0.024*"cable"'),
 (2,
  '0.079*"male" + 0.072*"mother" + 0.038*"matt" + 0.020*"honda" + '
  '0.020*"intelligent" + 0.011*"confine" + 0.009*"eliot" + 0.009*"gene" + '
  '0.006*"nonetheless" + 0.006*"typo"'),
 (3,
  '0.104*"server" + 0.068*"patient" + 0.061*"community" + 0.046*"medical" + '
  '0.046*"doctor" + 0.039*"archive" + 0.027*"medicine" + 0.025*"poster" + '
  '0.024*"hospital" + 0.019*"physician"'),
 (4,
  '0.059*"space" + 0.048*"research" + 0.027*"science" + 0.025*"center" + '
  '0.025*"tool" + 0.025*"development" + 0.023*"technology" + 0.021*"level" + '
  '0.021*"object" + 0.019*"input"'),
 (5,
  '0.044*"translation" + 0.038*"tor" + 0.029*"bos" + 0.028*"van" + 

In [24]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis

D:\Anaconda\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0      0.377968  0.001662       1        1  42.726292
9      0.344953  0.096356       2        1  13.326714
16     0.278405 -0.271078       3        1  13.015586
19     0.274554 -0.162222       4        1   8.798871
12     0.205468  0.221051       5        1   4.927341
14     0.164424  0.228413       6        1   4.534254
13    -0.038682  0.019176       7        1   4.093018
4      0.048792 -0.031529       8        1   1.841117
10    -0.091878  0.000715       9        1   1.404796
1     -0.052179 -0.078822      10        1   1.172396
8     -0.067233 -0.027401      11        1   0.734966
6     -0.120878 -0.004786      12        1   0.689575
7     -0.110392  0.009225      13        1   0.657851
17    -0.138342 -0.002432      14        1   0.572648
3     -0.157573  0.001355      15        1   0.428998
5     -0.176276  0.000091      16        1   0.340344
18    -0.168790 -0.003011      17        1   0.315224
15    -0.191382  0.000951      18        1   0.157470
11    -0.188001  0.000910      19        1   0.151346
2     -0.192959  0.001376      20        1   0.111189, topic_info=      Category          Freq           Term         Total  loglift  logprob
term                                                                       
29     Default  31720.000000           line  31720.000000  30.0000  30.0000
40     Default  28320.000000   organization  28320.000000  29.0000  29.0000
201    Default  37172.000000            not  37172.000000  28.0000  28.0000
4612   Default   9010.000000             ax   9010.000000  27.0000  27.0000
105    Default  12496.000000     university  12496.000000  26.0000  26.0000
283    Default  25630.000000          write  25630.000000  25.0000  25.0000
37     Default  10975.000000      nntp_post  10975.000000  24.0000  24.0000
296    Default  18718.000000        article  18718.000000  23.0000  23.0000
20     Default   9888.000000           host   9888.000000  22.0000  22.0000
209    Default  14229.000000         people  14229.000000  21.0000  21.0000
285    Default   8210.000000           year   8210.000000  20.0000  20.0000
282    Default  27727.000000          would  27727.000000  19.0000  19.0000
259    Default   9070.000000         system   9070.000000  18.0000  18.0000
322    Default   3942.000000          drive   3942.000000  17.0000  17.0000
658    Default   4000.000000              _   4000.000000  16.0000  16.0000
2138   Default   3362.000000            key   3362.000000  15.0000  15.0000
1534   Default   6765.000000         window   6765.000000  14.0000  14.0000
298    Default  19304.000000             be  19304.000000  13.0000  13.0000
385    Default  11696.000000            use  11696.000000  12.0000  12.0000
300    Default   3105.000000           book   3105.000000  11.0000  11.0000
156    Default  23798.000000             do  23798.000000  10.0000  10.0000
17     Default   4883.000000           game   4883.000000   9.0000   9.0000
231    Default  15810.000000            say  15810.000000   8.0000   8.0000
170    Default   4023.000000     government   4023.000000   7.0000   7.0000
412    Default   5637.000000        program   5637.000000   6.0000   6.0000
71     Default   2877.000000           card   2877.000000   5.0000   5.0000
227    Default   6596.000000          reply   6596.000000   4.0000   4.0000
1964   Default   5217.000000           file   5217.000000   3.0000   3.0000
33     Default   4161.000000            man   4161.000000   2.0000   2.0000
1667   Default   3846.000000           team   3846.000000   1.0000   1.0000
...        ...           ...            ...           ...      ...      ...
4199   Topic20     18.736317    nonetheless     20.012768   6.7358  -5.0384
3671   Topic20     17.774084           typo     19.050535   6.7323  -5.0911
811    Topic20      5.231698           karl      6.508149   6.5834  -6.3141
3668   Topic20      5.088010     

In [25]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -15.085717219922694

Coherence Score:  0.4783302228453776
